In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import seaborn as sns
import scipy
import warnings
warnings.filterwarnings('ignore')

In [2]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

### Интерпретация моделей

В мире анализа данных очень важным качеством модели является её интерпретируемость. Многие бизнес решения склоняются в пользу более простых моделей с меньшим качеством, ввиду лучшей интерпретируемости таких моделей. Если модель не поддаётся интерпретации, то её с меньшей охотой внедряют в продакшн, ведь нельзя проанализировать, на каких объектах эта модель поведёт анамально и почему она так себя повела. 

Однако и для сложных нелинейных моделей (деревья решений, нейронные сети и т.д.) существуют способы интерпретации и анализа их работы. 

Начнём с простейших линейных моделей - линейная и логистическая регрессии.

<font color='red'>Линейная регрессия</font> это модель вида: $y = w_{0} + w_{1} \cdot x_{1} + w_{2} \cdot x_{2} + \dots + w_{d} \cdot x_{d}, \ y \in \mathbb{R}$. Линейной она называется потому, что эта модель <b><i>линейная по параметрам ($w_{k}$)</i></b>, то есть модель $y = w_{0} + w_{1} \cdot x_{1}^2 + w_{2} \cdot x_{1}^3$ тоже будет называться линейной регрессией.

Её любят статистики за простую интерпретируемость. 

Рассмотрим на примере.

Пусть есть модель зависимости стоимости квартиры (млн. Руб.) от различных характеристик квартиры: 

$$price = 3 + 2 \cdot n\_rooms + 0.3 \cdot metro\_dist - 0.1 \cdot floor$$

Тогда коэффициент при n\_rooms можно проинтерпретировать так: 

При прочих равных условиях (а именно, если мы будем рассматривать множество квартир, отличающихся только количеством комнат), при увеличении числа комнат в квартире на 1 общая цена квартиры растёт <b>в среднем</b> на 2 млн. руб. 

При floor: При прочих равных условиях, при этажа на котором располагается квартира на 1 общая цена квартиры падает <b>в среднем</b> на 100 тыс. руб. (0.1 млн. руб.) 

Рассмотрим вариации этой модели и как будет меняться интерпретация в зависимости от вида линейной регрессии:

1) $y = w \cdot \ln x$

При прочих равных условиях при увеличении переменной x на $\boxed{1 \%}$ переменная y меняется на $\boxed{w \cdot 100 \ единиц}$

2) $\ln y = w \cdot x$

При прочих равных условиях при увеличении переменной x на $\boxed{1 \ единицу}$ переменная y меняется на $\boxed{(e^{w} - 1) \cdot 100\%}$

3) $\ln y = w \cdot \ln x$

При прочих равных условиях при увеличении переменной x на $\boxed{1\%}$ переменная y меняется на $\boxed{w\%}$

Однако с линейной регрессией нужно умень оценивать адекватность модели для того, чтобы можно было проводить такую интерпретацию коэффициентов. В случае наличиня [мультиколлинеарности](https://ru.wikipedia.org/wiki/Мультиколлинеарность) в модели оценки теряют интерпретируемость.

Рассмотрим пример.

Пусть была оценена линейная регрессия со следующими параметрами

$$y = 3 + 5 \cdot x  - 3 \cdot w + 15 \cdot z$$

Но пусть так же известно, что переменные x и w связаны соотношением: $z = 2 \cdot x - 7 \cdot w \Rightarrow z - 2 \cdot x + 7 \cdot w = 0$

Тогда: $y = 3 + 5 \cdot x  - 3 \cdot w + 15 \cdot z + 0 = 3 + 5 \cdot x  - 3 \cdot w + 15 \cdot z + (z - 2 \cdot x + 7 \cdot w) = 3 + 3 \cdot x + 4 \cdot w + 16 \cdot z$

Получаем совсем другие коэффициенты перед признаками, однако само уравнение по прежнему точно так же описывает игрек